In [9]:
import numpy as np
import os
from mido import MidiFile, MidiTrack, Message
import mido
from random import randrange
import NoteBins
from sklearn.cluster import KMeans

In [10]:
paths = []
songs = []
for r, d, f in os.walk(r"C:\Users\Vee\Desktop\python\timing\songs"):
    for file in f:
        if '.mid' in file:
            paths.append(os.path.join(r, file))

for path in paths:
    mid = MidiFile(path, type = 1)
    songs.append(mid)

In [11]:
notes = []
dataset = []
timing = []

In [12]:
#for each in midi object in list of songs
for i in range(len(songs)):
    #for each note in midi object
    for msg in songs[i]:
        if not msg.is_meta:
            if (msg.type == 'note_on'):
                notes.append([1,msg.note,int(msg.time*960)])
            elif(msg.type == 'note_off'):
                notes.append([0,msg.note,int(msg.time*960)])
    count = 0
    dict = {}
    for i in range(len(notes)):
        if not (notes[i][1] in dict):
            dict[notes[i][1]] = count
            notes[i][1] = count
            count += 1
        else:
            notes[i][1] = dict.pop(notes[i][1])
    timing = []
    for i in range(len(notes)):
        if(notes[i][1] < 16):
            timing.append(notes[i])
    timing[0][2] = 0
    dataset.append(timing)
    timing = []
    notes = []

In [13]:
dataset = np.array(dataset)
dataset.shape

(6, 32, 3)

In [14]:
final = []
for z in range(len(dataset)):
    length = []
    for i in range(len(dataset[z])):
        if(dataset[z][i][0] == 0):
            length.append(dataset[z][i][2])
    length = np.array(length)
    length = length.reshape(len(length),1)
    kmeans = KMeans(n_clusters=4, random_state=2).fit(length)
    x = kmeans.labels_
    clust = [-1,-1,-1,-1,-1]
    for i in range(len(x)):
        if(x[i] == 0):
            clust[0] = length[i][0]
        elif(x[i]==1):
            clust[1] = length[i][0]
        elif(x[i]==2):
            clust[2] = length[i][0]
        elif(x[i]==3):
            clust[3] = length[i][0]
        elif(x[i]==4):
            clust[4] = length[i][0]
    orig = np.array(clust)
    clust.sort()
    new = [-1,-1,-1,-1,-1]
    for i in range(len(clust)):
        for j in range(len(orig)):
            if(clust[i] == orig[j]):
                new[i] = j
    for i in range(len(new)):
        for j in range(len(x)):
            if(new[i] == x[j]):
                if(i == 0):
                    length[j][0] = 0
                elif(i == 1):
                    length[j][0] = 205
                elif(i == 2):
                    length[j][0] = 410
                elif(i == 3):
                    length[j][0] = 615
                elif(i == 4):
                    length[j][0] = 810
    count = 0
    for i in range(len(dataset[z])):
        if(dataset[z][i][0] == 0):
            dataset[z][i][2] = length[count]
            count+=1
    final.append(dataset[z])

In [15]:
final[1]

array([[  1,   0,   0],
       [  0,   0, 410],
       [  1,   1,   2],
       [  0,   1, 410],
       [  1,   2,   2],
       [  0,   2, 410],
       [  1,   3, 162],
       [  0,   3, 410],
       [  1,   4,   2],
       [  0,   4, 810],
       [  1,   5,   2],
       [  0,   5, 615],
       [  1,   6, 322],
       [  0,   6, 410],
       [  1,   7,   2],
       [  0,   7, 410],
       [  1,   8,   2],
       [  0,   8, 410],
       [  1,   9, 162],
       [  0,   9, 410],
       [  1,  10,   2],
       [  0,  10, 810],
       [  1,  11,   2],
       [  0,  11, 810],
       [  1,  12, 110],
       [  0,  12, 410],
       [  1,  13,   1],
       [  0,  13, 410],
       [  1,  14,  11],
       [  1,  15, 147],
       [  0,  14, 205],
       [  0,  15, 810]])

In [16]:
np.save(r'C:\Users\Vee\Desktop\python\timing\timing.npy',final)